Make plots comparing inferred dark HI column density with dust column density from IR SED fitting (Utomo+2019) and dust extinction from HST (Dalcanton+2015; M31 only).

We also use CO maps (M33 from Druard+2014; M31 from Nieten+2006) to remove regions where the dust column density is dominated by the molecular component (and has an additional conversion factor adding to the scatter).

In [ ]:

%matplotlib inline

from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
import os
import numpy as np
import matplotlib.pyplot as plt
from spectral_cube import Projection
from scipy import ndimage as nd
from astropy.stats import histogram as astro_hist
from tqdm import tqdm
from corner import hist2d
from scipy import ndimage as nd

from pathlib import Path

osjoin = os.path.join

repo_path = os.path.expanduser("~/ownCloud/project_code/ThickHIFitting/")

figures_path_png = osjoin(repo_path, "figures/png")
figures_path_pdf = osjoin(repo_path, "figures/pdf")


def save_figure(fig, plot_name, **kwargs):
    fig.savefig(f"{figures_path_pdf}/{plot_name}.pdf", **kwargs)
    fig.savefig(f"{figures_path_png}/{plot_name}.png", **kwargs)


paths_script = os.path.join(repo_path, "paths.py")
exec(compile(open(paths_script, "rb").read(), paths_script, 'exec'))

plotstyle_script = os.path.join(repo_path, "plotting_styles.py")
exec(compile(open(plotstyle_script, "rb").read(), plotstyle_script, 'exec'))

model_script = os.path.join(repo_path, "gaussian_model.py")
exec(compile(open(model_script, "rb").read(), model_script, 'exec'))

thickHI_model_script = os.path.join(repo_path, "thickHI_model.py")
exec(compile(open(thickHI_model_script, "rb").read(),
             thickHI_model_script, 'exec'))


In [ ]:
save_figures = False

In [ ]:
# Load in the moment 0 maps from the data and the dark HI intensity maps

m31_mom0 = Projection.from_hdu(fits.open(fifteenA_HI_BCtaper_wEBHIS_HI_file_dict['Moment0'])).to(u.K * u.km / u.s)

print(fifteenA_HI_BCtaper_wEBHIS_HI_file_dict['Moment0'])

m33_mom0_name = "M33_14B-088_HI.clean.image.GBT_feathered.pbcov_gt_0.5_masked.moment0_Kkms.fits"
m33_mom0 = Projection.from_hdu(fits.open(fourteenB_HI_data_wGBT_path(m33_mom0_name))).to(u.K * u.km / u.s)

m31_darkhi_hdu_name = fifteenA_HI_BCtaper_04kms_data_wEBHIS_path("individ_simplethick_HI_fits_5kms_centlimit_darkHI.fits")
m31_darkhi = Projection.from_hdu(fits.open(m31_darkhi_hdu_name)[0])
m31_residhi = Projection.from_hdu(fits.open(m31_darkhi_hdu_name)[3])

print(len(fits.open(m31_darkhi_hdu_name)))

m33_darkhi_hdu_name = fourteenB_HI_data_wGBT_path("individ_simplethick_HI_fits_5kms_centlimit_darkHI.fits")
m33_darkhi = Projection.from_hdu(fits.open(m33_darkhi_hdu_name)[0])
m33_residhi = Projection.from_hdu(fits.open(m33_darkhi_hdu_name)[3])
  
print(m31_mom0.beam)
print(m33_mom0.beam)
    
# Make "opaque-corrected" values.

# m31_opcorr_mom0 = m31_darkhi + (m31_mom0 - m31_residhi)
# m33_opcorr_mom0 = m33_darkhi + (m33_mom0 - m33_residhi)
# m31_opcorr_mom0 = m31_darkhi + m31_residhi
# m33_opcorr_mom0 = m33_darkhi + m33_residhi
m31_opcorr_mom0 = m31_darkhi + m31_mom0
m33_opcorr_mom0 = m33_darkhi + m33_mom0

In [ ]:
fig = plt.figure(figsize=(22, 16))

plt.subplot(241)
plt.imshow(np.log10(m31_opcorr_mom0.value), origin='lower')
plt.colorbar()
plt.subplot(242)
plt.imshow(np.log10(m31_mom0.value), origin='lower')
plt.colorbar()
plt.subplot(243)
plt.imshow(m31_darkhi.value, origin='lower')
plt.colorbar()
plt.subplot(244)
plt.imshow(m31_residhi.value, origin='lower')
plt.colorbar()

plt.subplot(245)
plt.imshow(m33_opcorr_mom0.value, origin='lower', vmax=1300)
plt.colorbar()
plt.subplot(246)
plt.imshow(m33_mom0.value, origin='lower')
plt.colorbar()
plt.subplot(247)
plt.imshow(m33_darkhi.value, origin='lower')
plt.colorbar()
plt.subplot(248)
plt.imshow(m33_residhi.value, origin='lower',) # vmax=1300, vmin=0)
plt.colorbar()


In [ ]:
fig = plt.figure(figsize=(22, 16))


plt.subplot(121)
plt.imshow(m31_opcorr_mom0.value - m31_mom0.value, origin='lower')
plt.colorbar()
plt.subplot(122)
plt.imshow(m33_opcorr_mom0.value - m33_mom0.value, origin='lower')
plt.colorbar()


In [ ]:
# Load in various dust maps. Dust column density from Utomo+19. Extinction map (M31) from Dalcanton+15

dust_path = Path("/home/ekoch/bigdata/ekoch/Utomo19_LGdust/")

dust_beam_m33 = Beam(41.0 * u.arcsec)

# m33_dust_hdu = fits.open(dust_path / "M33" / "m33_dust.surface.density_FB.beta=1.8_gauss41.0_regrid_bksub.fits" )
m33_dust_hdu = fits.open(dust_path / "idas_dust_models" / "m33/res_167pc_BE" / "m33_dust.surface.density_BE.beta=2.0_gauss41.0_regrid_bksub.fits.gz" )
# m33_dust_hdu = fits.open(dust_path / "idas_dust_models" / "m33/res_167pc_BE" / "m33_dust.surface.density.max_BE.beta=2.0_gauss41.0_regrid_bksub.fits.gz" )

m33_celest_hdr = WCS(m33_dust_hdu[0].header).celestial.to_header()
del m33_celest_hdr['MJD-OBS']

m33_dust_coldens = Projection.from_hdu(fits.PrimaryHDU(m33_dust_hdu[0].data[0], m33_celest_hdr))
m33_dust_coldens_err = Projection.from_hdu(fits.PrimaryHDU(m33_dust_hdu[0].data[2], m33_celest_hdr))

m33_dust_coldens = m33_dust_coldens.with_beam(dust_beam_m33)
m33_dust_coldens_err = m33_dust_coldens_err.with_beam(dust_beam_m33)

m33_dust_coldens_err = m33_dust_coldens_err[nd.find_objects(np.isfinite(m33_dust_coldens_err))[0]]
m33_dust_coldens = m33_dust_coldens[nd.find_objects(np.isfinite(m33_dust_coldens))[0]]

assert m33_dust_coldens.shape == m33_dust_coldens_err.shape

# m31_dust_hdu = fits.open(dust_path / "M31" / "m31_dust.surface.density_FB.beta=1.8_gauss46.3_regrid_bksub.fits" )
m31_dust_hdu = fits.open(dust_path / "idas_dust_models" / "m31/res_167pc_BE" / "m31_dust.surface.density_BE.beta=2.0_gauss46.3_regrid_bksub.fits.gz")
# m31_dust_hdu = fits.open(dust_path / "idas_dust_models" / "m31/res_167pc_BE" / "m31_dust.surface.density.max_BE.beta=2.0_gauss46.3_regrid_bksub.fits.gz")

m31_celest_hdr = WCS(m31_dust_hdu[0].header).celestial.to_header()
# del m31_celest_hdr['MJD-OBS']

dust_beam_m31 = Beam(46.3 * u.arcsec)

m31_dust_coldens = Projection.from_hdu(fits.PrimaryHDU(m31_dust_hdu[0].data[0], m31_celest_hdr))
m31_dust_coldens_err = Projection.from_hdu(fits.PrimaryHDU(m31_dust_hdu[0].data[2], m31_celest_hdr))

m31_dust_coldens = m31_dust_coldens.with_beam(dust_beam_m31)
m31_dust_coldens_err = m31_dust_coldens_err.with_beam(dust_beam_m31)

m31_dust_coldens_err = m31_dust_coldens_err[nd.find_objects(np.isfinite(m31_dust_coldens_err))[0]]
m31_dust_coldens = m31_dust_coldens[nd.find_objects(np.isfinite(m31_dust_coldens))[0]]

assert m31_dust_coldens.shape == m31_dust_coldens_err.shape

# M31 dust extinction

ext_path = Path("/home/ekoch/bigdata/ekoch/M31/PHAT/phat_dust_maps/")

# m31_dust_ext = Projection.from_hdu(fits.open(ext_path / "phat_dalcanton.AV.fits"))
# m31_dust_ext_err = Projection.from_hdu(fits.open(ext_path / "phat_dalcanton.AVerr.fits"))
m31_dust_ext = Projection.from_hdu(fits.open(ext_path / "phat_dalcanton.meanAV.fits"))
m31_dust_ext_err = Projection.from_hdu(fits.open(ext_path / "phat_dalcanton.meanAVerr.fits"))

# M31 CO

m31_co_mom0_hdu = fits.open(Path("/home/ekoch/bigdata/ekoch/M31/IRAM30m/") / "m31_iram_Kkms.fits")
m31_co_mom0 = Projection.from_hdu(m31_co_mom0_hdu)
m31_co_mom0._beam = Beam(23 * u.arcsec)

m31_co_mom0_smooth = m31_co_mom0.convolve_to(m31_dust_coldens.beam)
m31_co_mom0_rep = m31_co_mom0_smooth.reproject(m31_dust_coldens.header)

# M33 CO

# m33_co_mom0 = Projection.from_hdu(fits.open(Path("/home/ekoch/bigdata/ekoch/M33/co21/14B-088") / "m33.co21_iram.14B-088_HI.mom0.fits"))
m33_co_mom0 = Projection.from_hdu(fits.open(Path("/home/ekoch/bigdata/ekoch/M33/co21/14B-088") / "m33.co21_iram.14B-088_HI.mom0.fits"))
m33_co_mom0 = m33_co_mom0.to(u.K * u.km / u.s)

m33_co_mom0_smooth = m33_co_mom0.convolve_to(m33_dust_coldens.beam, nan_treatment='fill')
m33_co_mom0_rep = m33_co_mom0_smooth.reproject(m33_dust_coldens.header)


In [ ]:
m31_co_mom0_rep.quicklook()
m33_co_mom0_rep.quicklook()

In [ ]:
m33_dust_coldens.quicklook()
m33_dust_coldens_err.quicklook()

In [ ]:
# m31_dust_coldens.quicklook()
# m31_dust_coldens_err.quicklook()

(m31_dust_coldens / m31_dust_coldens_err).quicklook()

Define dust masks to avoid noisy edges. These will set the extents of the maps the will be considered in the comparison.

In [ ]:
m33_coldens_mask = nd.binary_erosion(np.isfinite(m33_dust_coldens_err),
                                     structure=np.ones((3, 3)),
                                     iterations=10)

# m33_coldens_sn_mask = (m33_dust_coldens / m33_dust_coldens_err).value > 2.
m33_coldens_sn_mask = (m33_dust_coldens / m33_dust_coldens_err).value > 0.1

plt.imshow(m33_dust_coldens.value, origin='lower')
plt.contour(m33_coldens_mask, colors='g', levels=[0.5])
plt.contour(m33_coldens_sn_mask, colors='c', levels=[0.5])

m33_coldens_mask = np.logical_and(m33_coldens_mask, m33_coldens_sn_mask)

In [ ]:
m31_coldens_mask = nd.binary_erosion(np.isfinite(m31_dust_coldens_err),
                                     structure=np.ones((3, 3)),
                                     iterations=11)

# m31_coldens_sn_mask = (m31_dust_coldens / m31_dust_coldens_err).value > 2.
m31_coldens_sn_mask = (m31_dust_coldens / m31_dust_coldens_err).value > 0.1

plt.imshow(m31_dust_coldens.value, origin='lower')
plt.contour(m31_coldens_mask, colors='g', levels=[0.5])
plt.contour(m31_coldens_sn_mask, colors='c', levels=[0.5])

m31_coldens_mask = np.logical_and(m31_coldens_mask, m31_coldens_sn_mask)

In [ ]:
fig, axs = plt.subplots(1, 2, sharex=True, sharey=True)

axs[0].imshow(m31_dust_ext.value, origin='lower')
axs[1].imshow(m31_dust_ext_err.value, origin='lower')

Make CO masks for later plots that will exclude H2 regions.

We define a lower threshold of 1.5 Msol pc-2 to remove. We make an inclination correction for each galaxy and then use a CO conversion factor of 4.8 Msol pc-2 / (K km s-1) for CO(1-0) and a line ratio of CO(2-1)/CO(1-0)=0.7 for M33 (Druard+2014).

In [ ]:
# intint_min = 2.5 / (4.8 * np.cos(77 * u.deg)).value
intint_min = 1.5 / (4.8 * np.cos(77 * u.deg)).value

print(intint_min)


m31_co_mask = m31_co_mom0_rep.value > intint_min

plt.imshow(m31_co_mom0_rep.value, origin='lower')
plt.contour(m31_co_mask, colors='g')

In [ ]:
# intint_min_co21 = 2.5 / ((4.8 / 0.7) * np.cos(56 * u.deg)).value
intint_min_co21 = 1.5 / ((4.8 / 0.7) * np.cos(56 * u.deg)).value

print(intint_min_co21)

m33_co_mask = m33_co_mom0_rep.value > intint_min_co21

plt.imshow(m33_co_mom0_rep.value, origin='lower')
plt.colorbar()
plt.contour(m33_co_mask, colors='g')

The extinction map has a square PSF based on the modeling used to derive the quantities. We use `photutils.create_matching_kernels` so we can convolve to a Gaussian PSF equivalent to the other maps. Because the Gaussian beam is so much larger, it turns out that doing this the "correct" way doesn't make too much of a difference vs. assuming a pencil beam equal to 1 pixel. But it would matter if the Gaussian was closer to the pixel scale.

In [ ]:
# Get pixscale in deg for the extinction maps.

from astropy.wcs.utils import proj_plane_pixel_scales
pixscale_ext = proj_plane_pixel_scales(m31_dust_ext.wcs)[0] * u.deg

print(pixscale_ext.to(u.arcsec))

In [ ]:
# Define the extinction beam and transformations to a 2D Gaussian

import photutils

from astropy.modeling.models import Box2D

m31_ext_beam = Box2D(1, x_0=25, y_0=25)

ys, xs = np.mgrid[0:51, 0:51]

m31_ext_kernel = m31_ext_beam(ys, xs)
print(m31_ext_kernel.sum())


# HI beam

m31_hi_beam = m31_mom0.beam

m31_hi_kernel = m31_hi_beam.as_kernel(pixscale=pixscale_ext, x_size=51, y_size=51)
print(m31_hi_kernel.array.sum())

# window = photutils.TopHatWindow(0.35)
m31_hi_ext_match_kernel = photutils.create_matching_kernel(m31_ext_kernel, m31_hi_kernel,
                                                           window=None)
plt.subplot(121)
plt.imshow(m31_hi_ext_match_kernel, vmax=0.001)
plt.colorbar()

# Looks good. Now for the dust.

m31_dust_beam = m31_dust_coldens.beam

m31_dust_kernel = m31_dust_beam.as_kernel(pixscale=pixscale_ext, x_size=51, y_size=51)

m31_dust_ext_match_kernel = photutils.create_matching_kernel(m31_ext_kernel, m31_dust_kernel,
                                                             window=None)

plt.subplot(122)
plt.imshow(m31_dust_ext_match_kernel, vmax=0.001)
plt.colorbar()



In [ ]:
# Try reprojecting the M31 data to match the dust emission col. density:
# Convolve to the HI and dust beams.

from astropy.convolution import convolve_fft

m31_ext_hi_data = convolve_fft(m31_dust_ext.value, m31_hi_ext_match_kernel)

m31_ext_dust_data = convolve_fft(m31_dust_ext.value, m31_dust_ext_match_kernel)

m31_ext_matchhi = Projection.from_hdu(fits.PrimaryHDU(m31_ext_hi_data, m31_dust_ext.header))
m31_ext_matchdust = Projection.from_hdu(fits.PrimaryHDU(m31_ext_dust_data, m31_dust_ext.header))

m31_ext_matchhi_rep = m31_ext_matchhi.reproject(m31_mom0.header)
m31_ext_matchdust_rep = m31_ext_matchdust.reproject(m31_dust_coldens.header)


Check the PSF matched maps:

In [ ]:

plt.subplot(221)
plt.imshow(m31_ext_hi_data, origin='lower')
plt.subplot(222)
plt.imshow(m31_ext_dust_data, origin='lower')

plt.subplot(223)
plt.imshow(m31_ext_matchhi_rep.value, origin='lower')
plt.subplot(224)
plt.imshow(m31_ext_matchdust_rep.value, origin='lower')


In [ ]:
plt.subplot(121)
plt.imshow(m31_mom0.value, origin='lower')
plt.contour(m31_ext_matchhi_rep.value, levels=[1.0, 2.0], colors='w')

plt.subplot(122)
plt.imshow(m31_dust_coldens.value, origin='lower')
plt.contour(m31_ext_matchdust_rep.value, levels=[1.0, 2.0], colors='w')


In [ ]:
# Also smooth and reproject the HI maps to match the dust emission

m31_opcorr_mom0_smooth = m31_opcorr_mom0.convolve_to(m31_dust_coldens.beam)
m31_mom0_smooth = m31_mom0.convolve_to(m31_dust_coldens.beam)

m31_opcorr_mom0_matchdust = m31_opcorr_mom0_smooth.reproject(m31_dust_coldens.header)
m31_mom0_matchdust = m31_mom0_smooth.reproject(m31_dust_coldens.header)

m33_opcorr_mom0_smooth = m33_opcorr_mom0.convolve_to(m33_dust_coldens.beam)
m33_mom0_smooth = m33_mom0.convolve_to(m33_dust_coldens.beam)

m33_opcorr_mom0_matchdust = m33_opcorr_mom0_smooth.reproject(m33_dust_coldens.header)
m33_mom0_matchdust = m33_mom0_smooth.reproject(m33_dust_coldens.header)



We should be ready for some plots now!

* M31 and M33 dust vs. HI surface density
  * all points (to check)
  * CO masked (for paper)
  * Show dust-to-gas ratios needed to account for "opcorr" values
* M31 extinction vs HI integrated intensity
  * CO masked (for paper)



Comparisons with matching the extinction map.

In [ ]:
hi_ext_mask = np.logical_and(np.isfinite(m31_mom0), np.isfinite(m31_ext_matchhi_rep))
hi_ext_mask_opcorr = np.logical_and(np.isfinite(m31_opcorr_mom0), np.isfinite(m31_ext_matchhi_rep))

fig, axs = plt.subplots(1, 2)

# plt.scatter(m31_mom0[hi_ext_mask], m31_ext_matchhi_rep[hi_ext_mask])
hist2d(m31_mom0.value[hi_ext_mask], m31_ext_matchhi_rep.value[hi_ext_mask], ax=axs[0])
hist2d(m31_opcorr_mom0.value[hi_ext_mask_opcorr], m31_ext_matchhi_rep.value[hi_ext_mask_opcorr], ax=axs[1])

In [ ]:
dust_ext_mask = np.logical_and(np.isfinite(m31_dust_coldens), np.isfinite(m31_ext_matchdust_rep))

# plt.scatter(m31_mom0[hi_ext_mask], m31_ext_matchhi_rep[hi_ext_mask])
hist2d(m31_dust_coldens.value[dust_ext_mask], m31_ext_matchdust_rep.value[dust_ext_mask])


Comparisons of HI and dust emission at matched resolution.

In [ ]:
default_figure()

fig = plt.figure(figsize=(22, 22))

plt.subplot(221)
plt.imshow(m31_dust_coldens.value, origin='lower')
plt.contour(m31_mom0_matchdust.value, colors='g', levels=5)

plt.subplot(222)
plt.imshow(m33_dust_coldens.value, origin='lower')
plt.contour(m33_mom0_matchdust.value, colors='g', levels=5)

plt.subplot(223)
plt.imshow(m31_dust_coldens.value, origin='lower')
plt.contour(m31_opcorr_mom0_matchdust.value, colors='g', levels=5)

plt.subplot(224)
plt.imshow(m33_dust_coldens.value, origin='lower')
plt.contour(m33_opcorr_mom0_matchdust.value, colors='g', levels=5)

In [ ]:
# plt.imshow(m31_dust_coldens.value, origin='lower')
plt.imshow(m31_mom0_matchdust.value, origin='lower')
plt.colorbar()
plt.contour(np.isfinite(m31_mom0_matchdust.value), colors='g', levels=[0.5])
plt.contour(m31_mom0_matchdust.value > 100, colors='g', levels=[0.5])

In [ ]:
# plt.imshow(m31_dust_coldens.value, origin='lower')
plt.imshow(m33_mom0_matchdust.value, origin='lower')
plt.colorbar()
plt.contour(np.isfinite(m33_mom0_matchdust.value), colors='g', levels=[0.5])
plt.contour(m33_mom0_matchdust.value > 100, colors='g', levels=[0.5])

In [ ]:
# Options for hist2d set in one place

data_kwargs = {'alpha': 0.3}

In [ ]:

twocolumn_figure()

hi_coldens_factor = 0.0196

m31_inc_corr = np.cos(77 * u.deg).value
m33_inc_corr = np.cos(56 * u.deg).value

# m31_hi_dust_mask = np.logical_and(np.isfinite(m31_mom0_matchdust), m31_coldens_mask)
m31_hi_dust_mask = np.logical_and(m31_mom0_matchdust.value > 50, m31_coldens_mask)

m33_hi_dust_mask = np.logical_and(m33_mom0_matchdust.value > 50, m33_coldens_mask)

# hi_ext_mask_opcorr = np.logical_and(np.isfinite(m31_opcorr_mom0), np.isfinite(m31_ext_matchhi_rep))

fig, axs = plt.subplots(2, 2)

print(np.nanmax(m31_dust_coldens.value[m31_hi_dust_mask] * m31_inc_corr))

hist2d(hi_coldens_factor * m33_mom0_matchdust.value[m33_hi_dust_mask] * m33_inc_corr,
       m33_dust_coldens.value[m33_hi_dust_mask] * m33_inc_corr, ax=axs[1, 0], data_kwargs=data_kwargs)
hist2d(hi_coldens_factor * m33_opcorr_mom0_matchdust.value[m33_hi_dust_mask] * m33_inc_corr,
       m33_dust_coldens.value[m33_hi_dust_mask] * m33_inc_corr, ax=axs[1, 1], data_kwargs=data_kwargs)

hist2d(hi_coldens_factor * m31_mom0_matchdust.value[m31_hi_dust_mask] * m31_inc_corr,
       m31_dust_coldens.value[m31_hi_dust_mask] * m31_inc_corr, ax=axs[0, 0], data_kwargs=data_kwargs)
hist2d(hi_coldens_factor * m31_opcorr_mom0_matchdust.value[m31_hi_dust_mask] * m31_inc_corr,
       m31_dust_coldens.value[m31_hi_dust_mask] * m31_inc_corr, ax=axs[0, 1],
       bins=[20, 20], data_kwargs=data_kwargs)

print(m31_mom0_matchdust.value[m31_hi_dust_mask].shape)
print(m33_mom0_matchdust.value[m33_hi_dust_mask].shape)

bbox_props = dict(boxstyle='round', facecolor='white', alpha=1., edgecolor='k')

xp, yp = 0.1, 0.95

axs[0, 0].text(xp, yp, "M31", # \nOptically-thin",
               transform=axs[0, 0].transAxes,
               horizontalalignment='center',
               verticalalignment='top',
               bbox=bbox_props, fontsize=15)
axs[0, 1].text(xp, yp, "M31", # \nInferred opaque correction",
               transform=axs[0, 1].transAxes,
               horizontalalignment='center',
               verticalalignment='top',
               bbox=bbox_props, fontsize=15)

axs[1, 0].text(xp, yp, "M33", # \nOptically-thin",
               transform=axs[1, 0].transAxes,
               horizontalalignment='center',
               verticalalignment='top',
               bbox=bbox_props, fontsize=15)
axs[1, 1].text(xp, yp, "M33", # \nInferred opaque correction",
               transform=axs[1, 1].transAxes,
               horizontalalignment='center',
               verticalalignment='top',
               bbox=bbox_props, fontsize=15)

[ax.grid(True) for ax in axs.ravel()]

xlims = np.array([[60, 320], [25, 90]])
gdrs = [125, 200, 300, 1000]

lstyles = [linestyles['solid'],
           linestyles['dotted'],
           linestyles['dashed'],
           linestyles['dashdotted'],
           linestyles['densely dashed']]

for i, (ax, xl) in enumerate(zip(axs.ravel(), xlims.ravel())):
    for gdr, lstyle in zip(gdrs, lstyles):
        im = ax.plot([0, xl], np.array([0, xl]) / gdr, label=f"GDR={gdr}",
                     linestyle=lstyle, linewidth=2)

# axs[0].plot([0, 65], np.array([0, 65]) / 100, 'g--', label='GDR=100')
# axs[0].plot([0, 65], np.array([0, 65]) / 200, 'b:', label='GDR=200')
# axs[0].plot([0, 65], np.array([0, 65]) / 300, 'r-.', label='GDR=300')
# axs[0].plot([0, 65], np.array([0, 65]) / 500, 'r-.', label='GDR=500')
# axs[0].plot([0, 65], np.array([0, 65]) / 1000, 'r-.', label='GDR=1000')

handles, labels = axs[0, 0].get_legend_handles_labels()

fig.legend(handles, labels, ncol=5, loc='upper center')
# axs[0].legend(frameon=True, ncol=2)

axs[1, 0].set_xlabel(r"$\Sigma_{\rm atomic}$ (M$_\odot$ pc$^{-2}$)")
axs[1, 1].set_xlabel(r"$\Sigma_{\rm atomic}$ (M$_\odot$ pc$^{-2}$)")

axs[1, 0].set_ylabel(r"$\Sigma_{\rm dust}$ (M$_\odot$ pc$^{-2}$)")
axs[0, 0].set_ylabel(r"$\Sigma_{\rm dust}$ (M$_\odot$ pc$^{-2}$)")

axs[0, 0].set_title("Optically-thin", fontsize=15)
axs[0, 1].set_title("With inferred opacity correction", fontsize=15)

In [ ]:
# Check the masks we're using:

plt.subplot(121)
plt.imshow(m31_mom0_matchdust.value, origin='lower')
plt.contour(m31_hi_dust_mask, colors='c', levels=[0.5])

plt.subplot(122)
plt.imshow(m33_mom0_matchdust.value, origin='lower')
plt.contour(m33_hi_dust_mask, colors='c', levels=[0.5])


Add in CO mask. This is what we use in the paper.

In [ ]:

twocolumn_figure()

hi_coldens_factor = 0.0196

m31_inc_corr = np.cos(77 * u.deg).value
m33_inc_corr = np.cos(56 * u.deg).value

# m31_hi_dust_mask = np.logical_and(np.isfinite(m31_mom0_matchdust), m31_coldens_mask)
m31_hi_dust_mask = np.logical_and(m31_mom0_matchdust.value > 50, m31_coldens_mask)
m31_hi_dust_mask = np.logical_and(m31_hi_dust_mask, ~m31_co_mask)

m33_hi_dust_mask = np.logical_and(m33_mom0_matchdust.value > 50, m33_coldens_mask)
m33_hi_dust_mask = np.logical_and(m33_hi_dust_mask, ~m33_co_mask)

# hi_ext_mask_opcorr = np.logical_and(np.isfinite(m31_opcorr_mom0), np.isfinite(m31_ext_matchhi_rep))

fig, axs = plt.subplots(2, 2, sharey=True)

hist2d(hi_coldens_factor * m31_mom0_matchdust.value[m31_hi_dust_mask] * m31_inc_corr,
       m31_dust_coldens.value[m31_hi_dust_mask] * m31_inc_corr,
       ax=axs[0, 0], data_kwargs=data_kwargs)
hist2d(hi_coldens_factor * m31_opcorr_mom0_matchdust.value[m31_hi_dust_mask] * m31_inc_corr,
       m31_dust_coldens.value[m31_hi_dust_mask] * m31_inc_corr,
       ax=axs[0, 1],
       bins=[20, 20], data_kwargs=data_kwargs)

hist2d(hi_coldens_factor * m33_mom0_matchdust.value[m33_hi_dust_mask] * m33_inc_corr,
       m33_dust_coldens.value[m33_hi_dust_mask] * m33_inc_corr,
       ax=axs[1, 0], data_kwargs=data_kwargs)
hist2d(hi_coldens_factor * m33_opcorr_mom0_matchdust.value[m33_hi_dust_mask] * m33_inc_corr,
       m33_dust_coldens.value[m33_hi_dust_mask] * m33_inc_corr,
       ax=axs[1, 1], data_kwargs=data_kwargs)

print(m31_mom0_matchdust.value[m31_hi_dust_mask].shape)
print(m33_mom0_matchdust.value[m33_hi_dust_mask].shape)

bbox_props = dict(boxstyle='round', facecolor='white', alpha=1., edgecolor='k')

xp, yp = 0.1, 0.9

axs[0, 0].text(xp, yp, "M31", # \nOptically-thin",
               transform=axs[0, 0].transAxes,
               horizontalalignment='center',
               verticalalignment='center',
               bbox=bbox_props, fontsize=15)
axs[0, 1].text(xp, yp, "M31", # \nInferred opaque correction",
               transform=axs[0, 1].transAxes,
               horizontalalignment='center',
               verticalalignment='center',
               bbox=bbox_props, fontsize=15)

axs[1, 0].text(xp, yp, "M33", # \nOptically-thin",
               transform=axs[1, 0].transAxes,
               horizontalalignment='center',
               verticalalignment='center',
               bbox=bbox_props, fontsize=15)
axs[1, 1].text(xp, yp, "M33", # \nInferred opaque correction",
               transform=axs[1, 1].transAxes,
               horizontalalignment='center',
               verticalalignment='center',
               bbox=bbox_props, fontsize=15)

[ax.grid(True) for ax in axs.ravel()]

# xlims = np.array([[60, 320], [25, 90]])
xlims = np.array([[60, 320], [60, 320]])
gdrs = [125, 200, 300, 1000]

lstyles = [linestyles['solid'],
           linestyles['dotted'],
           linestyles['dashed'],
#            linestyles['dashdotted'],
           linestyles['densely dashed']]

for i, (ax, xl) in enumerate(zip(axs.ravel(), xlims.ravel())):
    for gdr, lstyle in zip(gdrs, lstyles):
        im = ax.plot([0, xl], np.array([0, xl]) / gdr, label=f"GDR={gdr}",
                     linestyle=lstyle, linewidth=2)

# axs[0].plot([0, 65], np.array([0, 65]) / 100, 'g--', label='GDR=100')
# axs[0].plot([0, 65], np.array([0, 65]) / 200, 'b:', label='GDR=200')
# axs[0].plot([0, 65], np.array([0, 65]) / 300, 'r-.', label='GDR=300')
# axs[0].plot([0, 65], np.array([0, 65]) / 500, 'r-.', label='GDR=500')
# axs[0].plot([0, 65], np.array([0, 65]) / 1000, 'r-.', label='GDR=1000')

handles, labels = axs[0, 0].get_legend_handles_labels()

fig.legend(handles, labels, ncol=5, loc='upper center')
# axs[0].legend(frameon=True, ncol=2)

axs[1, 0].set_xlabel(r"$\Sigma_{\rm atomic}$ (M$_\odot$ pc$^{-2}$)")
axs[1, 1].set_xlabel(r"$\Sigma_{\rm atomic}$ (M$_\odot$ pc$^{-2}$)")

axs[1, 0].set_ylabel(r"$\Sigma_{\rm dust}$ (M$_\odot$ pc$^{-2}$)")
axs[0, 0].set_ylabel(r"$\Sigma_{\rm dust}$ (M$_\odot$ pc$^{-2}$)")

axs[0, 0].set_title("Optically-thin", fontsize=15)
axs[0, 1].set_title("With inferred opacity correction", fontsize=15)

plt.subplots_adjust(hspace=0.05, wspace=0.05)

if save_figures:
    save_figure(fig, "m31_m33_hi_dust_wGDR_COmasked", bbox_inches='tight', dpi=300)

In [ ]:
# Check the masks we're using:

plt.figure(figsize=(22, 12))

plt.subplot(221)
plt.imshow(m31_mom0_matchdust.value, origin='lower', vmin=0.)
plt.contour(m31_hi_dust_mask, colors='c', levels=[0.5])

plt.subplot(222)
plt.imshow(m33_mom0_matchdust.value, origin='lower', vmin=0.)
plt.contour(m33_hi_dust_mask, colors='c', levels=[0.5])

plt.subplot(223)
plt.imshow(m31_dust_coldens.value * m31_inc_corr, origin='lower', vmin=0.)
plt.colorbar()
plt.contour(m31_hi_dust_mask, colors='c', levels=[0.5])

plt.subplot(224)
plt.imshow(m33_dust_coldens.value * m33_inc_corr, origin='lower', vmin=0.)
plt.colorbar()
plt.contour(m33_hi_dust_mask, colors='c', levels=[0.5])

plt.tight_layout()

For paper version, add in a legend for the GDR lines.

In [ ]:

twocolumn_figure()

hi_coldens_factor = 0.0196

m31_inc_corr = np.cos(77 * u.deg).value
m33_inc_corr = np.cos(56 * u.deg).value

# m31_hi_dust_mask = np.logical_and(np.isfinite(m31_mom0_matchdust), m31_coldens_mask)
m31_hi_dust_mask = np.logical_and(m31_mom0_matchdust.value > 50, m31_coldens_mask)
m31_hi_dust_mask = np.logical_and(m31_hi_dust_mask, ~m31_co_mask)

m33_hi_dust_mask = np.logical_and(m33_mom0_matchdust.value > 50, m33_coldens_mask)
m33_hi_dust_mask = np.logical_and(m33_hi_dust_mask, ~m33_co_mask)

# hi_ext_mask_opcorr = np.logical_and(np.isfinite(m31_opcorr_mom0), np.isfinite(m31_ext_matchhi_rep))

fig, axs = plt.subplots(2, 2, sharex=True, sharey=True)

hist2d(hi_coldens_factor * m31_mom0_matchdust.value[m31_hi_dust_mask] * m31_inc_corr,
       m31_dust_coldens.value[m31_hi_dust_mask] * m31_inc_corr,
       ax=axs[0, 0], data_kwargs=data_kwargs,
      range=[(0., 52.), (0., 0.125)])
hist2d(hi_coldens_factor * m31_opcorr_mom0_matchdust.value[m31_hi_dust_mask] * m31_inc_corr,
       m31_dust_coldens.value[m31_hi_dust_mask] * m31_inc_corr,
       ax=axs[0, 1],
       bins=[20, 20], data_kwargs=data_kwargs,
      range=[(0., 52.), (0., 0.125)])

hist2d(hi_coldens_factor * m33_mom0_matchdust.value[m33_hi_dust_mask] * m33_inc_corr,
       m33_dust_coldens.value[m33_hi_dust_mask] * m33_inc_corr,
       ax=axs[1, 0], data_kwargs=data_kwargs,
       range=[(0., 52.), (0., 0.125)])
hist2d(hi_coldens_factor * m33_opcorr_mom0_matchdust.value[m33_hi_dust_mask] * m33_inc_corr,
       m33_dust_coldens.value[m33_hi_dust_mask] * m33_inc_corr,
       ax=axs[1, 1], data_kwargs=data_kwargs,
       range=[(0., 52.), (0., 0.125)])

print(m31_mom0_matchdust.value[m31_hi_dust_mask].shape)
print(m33_mom0_matchdust.value[m33_hi_dust_mask].shape)

bbox_props = dict(boxstyle='round', facecolor='white', alpha=1., edgecolor='k')

xp, yp = 0.85, 0.8

corrcoef = np.corrcoef(hi_coldens_factor * m31_mom0_matchdust.value[m31_hi_dust_mask] * m31_inc_corr,
                       m31_dust_coldens.value[m31_hi_dust_mask] * m31_inc_corr,)[0, 1]
axs[0, 0].text(xp, yp, f"M31\nr={corrcoef:.2f}", # \nOptically-thin",
               transform=axs[0, 0].transAxes,
               horizontalalignment='center',
               verticalalignment='center',
               bbox=bbox_props, fontsize=15)
corrcoef = np.corrcoef(hi_coldens_factor * m31_opcorr_mom0_matchdust.value[m31_hi_dust_mask] * m31_inc_corr,
                       m31_dust_coldens.value[m31_hi_dust_mask] * m31_inc_corr,)[0, 1]
axs[0, 1].text(xp, yp, f"M31\nr={corrcoef:.2f}", # \nInferred opaque correction",
               transform=axs[0, 1].transAxes,
               horizontalalignment='center',
               verticalalignment='center',
               bbox=bbox_props, fontsize=15)

corrcoef = np.corrcoef(hi_coldens_factor * m33_mom0_matchdust.value[m33_hi_dust_mask] * m33_inc_corr,
                       m33_dust_coldens.value[m33_hi_dust_mask] * m33_inc_corr)[0, 1]
axs[1, 0].text(xp, yp, f"M33\nr={corrcoef:.2f}", # \nOptically-thin",
               transform=axs[1, 0].transAxes,
               horizontalalignment='center',
               verticalalignment='center',
               bbox=bbox_props, fontsize=15)

corrcoef = np.corrcoef(hi_coldens_factor * m33_opcorr_mom0_matchdust.value[m33_hi_dust_mask] * m33_inc_corr,
                       m33_dust_coldens.value[m33_hi_dust_mask] * m33_inc_corr,)[0, 1]
axs[1, 1].text(xp, yp, f"M33\nr={corrcoef:.2f}", # \nInferred opaque correction",
               transform=axs[1, 1].transAxes,
               horizontalalignment='center',
               verticalalignment='center',
               bbox=bbox_props, fontsize=15)

[ax.grid(True) for ax in axs.ravel()]

# xlims = np.array([[60, 320], [25, 90]])
xlims = np.array([[60, 320], [60, 320]])
gdrs = [125, 200, 300, 1000]

lstyles = [linestyles['solid'],
           linestyles['dotted'],
           linestyles['dashed'],
#            linestyles['dashdotted'],
           linestyles['densely dashed']]

for i, (ax, xl) in enumerate(zip(axs.ravel(), xlims.ravel())):
    for gdr, lstyle in zip(gdrs, lstyles):
        im = ax.plot([0, xl], np.array([0, xl]) / gdr, label=f"GDR={gdr}",
                     linestyle=lstyle, linewidth=2)

# axs[0].plot([0, 65], np.array([0, 65]) / 100, 'g--', label='GDR=100')
# axs[0].plot([0, 65], np.array([0, 65]) / 200, 'b:', label='GDR=200')
# axs[0].plot([0, 65], np.array([0, 65]) / 300, 'r-.', label='GDR=300')
# axs[0].plot([0, 65], np.array([0, 65]) / 500, 'r-.', label='GDR=500')
# axs[0].plot([0, 65], np.array([0, 65]) / 1000, 'r-.', label='GDR=1000')

handles, labels = axs[0, 0].get_legend_handles_labels()

fig.legend(handles, labels, ncol=5, loc='upper center')
# axs[0].legend(frameon=True, ncol=2)

axs[1, 0].set_xlabel(r"$\Sigma_{\rm atomic}$ (M$_\odot$ pc$^{-2}$)")
axs[1, 1].set_xlabel(r"$\Sigma_{\rm atomic}$ (M$_\odot$ pc$^{-2}$)")

axs[1, 0].set_ylabel(r"$\Sigma_{\rm dust}$ (M$_\odot$ pc$^{-2}$)")
axs[0, 0].set_ylabel(r"$\Sigma_{\rm dust}$ (M$_\odot$ pc$^{-2}$)")

axs[0, 0].set_title("Optically-thin", fontsize=15)
axs[0, 1].set_title("With inferred opacity correction", fontsize=15)

plt.subplots_adjust(hspace=0.07, wspace=0.05)

if save_figures:
    save_figure(fig, "m31_m33_hi_dust_wGDR_COmasked_matchxrange", bbox_inches='tight', dpi=300)

In [ ]:
# Directly show the ratios:

m31_gdr = hi_coldens_factor * m31_mom0_matchdust.value[m31_hi_dust_mask] / m31_dust_coldens.value[m31_hi_dust_mask]

m33_gdr = hi_coldens_factor * m33_mom0_matchdust.value[m33_hi_dust_mask] / m33_dust_coldens.value[m33_hi_dust_mask]

fig, axs = plt.subplots(1, 2, sharex=True, sharey=True)

axs[0].set_title("M31")

axs[0].hist(np.log10(m31_gdr), bins=20, zorder=-10)

axs[0].axvline(np.log10(np.median(m31_gdr)), color='r', linestyle='--', zorder=-1, linewidth=3)
axs[0].axvline(np.log10(226.), color='k', linestyle=':', zorder=-1, linewidth=3)

axs[0].set_xlabel("log10 GDR (HI-to-dust)")

axs[1].set_title("M33")

axs[1].hist(np.log10(m33_gdr), bins=20, zorder=-10)

axs[1].axvline(np.log10(np.median(m33_gdr)), color='r', linestyle='--', zorder=-1, linewidth=3)
axs[1].axvline(np.log10(264.), color='k', linestyle=':', zorder=-1, linewidth=3)


print(f"{np.median(m31_gdr)} {np.median(m33_gdr)}")

axs[1].set_xlabel("log10 GDR (HI-to-dust)")

if save_figures:
    save_figure(fig, "m31_m33_gdr_histograms", bbox_inches='tight', dpi=300)

Plot against I-Da's median GDR values. These match now!

Next plot: M31 HI vs. extinction.

In [ ]:
# onecolumn_figure()
twocolumn_twopanel_figure()

fig, axs = plt.subplots(1, 2, sharey=True)

m31_av_mask = m31_ext_matchhi_rep.value > 0.5

m31_hi_av_mask = np.logical_and(m31_mom0.value > 50, m31_av_mask)
m31_hi_av_mask = np.logical_and(m31_hi_av_mask, np.isfinite(m31_opcorr_mom0))

hist2d(hi_coldens_factor * m31_mom0.value[m31_hi_av_mask] * m31_inc_corr,
       m31_ext_matchhi_rep.value[m31_hi_av_mask], # * m31_inc_corr,
       ax=axs[0], data_kwargs=data_kwargs)
hist2d(hi_coldens_factor * m31_opcorr_mom0.value[m31_hi_av_mask] * m31_inc_corr,
       m31_ext_matchhi_rep.value[m31_hi_av_mask], # * m31_inc_corr,
       ax=axs[1],
       bins=[40, 40], data_kwargs=data_kwargs)

[ax.grid(True) for ax in axs.ravel()]

axs[0].set_ylabel(r"A$_{\rm V}$")
axs[0].set_xlabel(r"$\Sigma_{\rm atomic}$ (M$_\odot$ pc$^{-2}$)")
axs[1].set_xlabel(r"$\Sigma_{\rm atomic}$ (M$_\odot$ pc$^{-2}$)")

axs[0].set_title("Optically-thin", fontsize=15)
axs[1].set_title("With inferred opacity correction", fontsize=15)

bbox_props = dict(boxstyle='round', facecolor='white', alpha=1., edgecolor='k')

xp, yp = 0.1, 0.9

axs[0].text(xp, yp, "M31", # \nOptically-thin",
            transform=axs[0].transAxes,
            horizontalalignment='center',
            verticalalignment='center',
            bbox=bbox_props, fontsize=15)


plt.subplots_adjust(hspace=0.05, wspace=0.05)

if save_figures:
    save_figure(fig, "m31_hi_Av", bbox_inches='tight', dpi=300)

In [ ]:
# onecolumn_figure()
twocolumn_twopanel_figure()

fig, axs = plt.subplots(1, 2, sharey=True, sharex=True)

m31_av_mask = m31_ext_matchhi_rep.value > 0.5

m31_hi_av_mask = np.logical_and(m31_mom0.value > 50, m31_av_mask)
m31_hi_av_mask = np.logical_and(m31_hi_av_mask, np.isfinite(m31_opcorr_mom0))

hist2d(hi_coldens_factor * m31_mom0.value[m31_hi_av_mask] * m31_inc_corr,
       m31_ext_matchhi_rep.value[m31_hi_av_mask], # * m31_inc_corr,
       ax=axs[0], data_kwargs=data_kwargs,
       range=[(0, 85), (0, 6)])
hist2d(hi_coldens_factor * m31_opcorr_mom0.value[m31_hi_av_mask] * m31_inc_corr,
       m31_ext_matchhi_rep.value[m31_hi_av_mask], # * m31_inc_corr,
       ax=axs[1],
       bins=[40, 40], data_kwargs=data_kwargs,
       range=[(0, 85), (0, 6)])

[ax.grid(True) for ax in axs.ravel()]

axs[0].set_ylabel(r"A$_{\rm V}$")
axs[0].set_xlabel(r"$\Sigma_{\rm atomic}$ (M$_\odot$ pc$^{-2}$)")
axs[1].set_xlabel(r"$\Sigma_{\rm atomic}$ (M$_\odot$ pc$^{-2}$)")

axs[0].set_title("Optically-thin", fontsize=15)
axs[1].set_title("With inferred opacity correction", fontsize=15)

bbox_props = dict(boxstyle='round', facecolor='white', alpha=1., edgecolor='k')

xp, yp = 0.1, 0.9

axs[0].text(xp, yp, "M31", # \nOptically-thin",
            transform=axs[0].transAxes,
            horizontalalignment='center',
            verticalalignment='center',
            bbox=bbox_props, fontsize=15)

plt.subplots_adjust(hspace=0.05, wspace=0.05)

if save_figures:
    save_figure(fig, "m31_hi_Av_matchxrange", bbox_inches='tight', dpi=300)